In [90]:
import os
import pyspark.mllib
import numpy as np
import matplotlib.pyplot
import seaborn as sns
import pyspark
import urllib

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext, SparkSession, Row
from pyspark.sql.types import *
from pyspark.sql.functions import *
from pyspark.sql import functions as func
from pyspark.ml.feature import OneHotEncoder, StringIndexer, VectorAssembler
from pyspark.mllib.regression import LinearRegressionWithSGD
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree
from IPython.display import display, HTML
import numpy as np
get_ipython().magic(u'pylab inline')


# import org.apache.spark._
# import org.apache.spark.sql._
# import org.apache.spark.ml._
# import org.apache.spark.sql.types._
# import org.apache.spark.storage.StorageLevel
# import org.apache.spark.sql.functions._
# import org.apache.spark.ml.feature.VectorAssembler
# import org.apache.hadoop.io.compress.GzipCodec

Populating the interactive namespace from numpy and matplotlib


In [7]:
### Initialize streaming context
conf = SparkConf()\
                .setMaster("local[2]")\
                .setAppName("MobileAnalytics")\
                .set("spark.executor.memory", "4g")\
                .set("spark.driver.memory", "4g")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")
sqlContext = SQLContext(sc)
spark = SparkSession.builder.appName("spark play").getOrCreate()

In [8]:
gender_age_train_csv = "data/gender_age_train.csv"
events_csv = "data/events.csv"
app_events_csv = "data/app_events.csv"
phone_brand_model_csv = "data/phone_brand_device_model_key.csv"
gender_age_test_csv = "data/gender_age_test.csv"
label_lookup_csv ="data/label_lookup.csv"

train_CSV ="data/train"
test_CSV ="data/test"

In [103]:
gender_age_train = spark.read.csv(gender_age_train_csv, header=True, mode="DROPMALFORMED", inferSchema='true', encoding="utf-8")\
                        .select("device_id","group")
    
# label_lookup = spark.read.csv(label_lookup_csv, header=True, mode="DROPMALFORMED", inferSchema='true', encoding="utf-8")
events = spark.read.csv(events_csv, header=True, mode="DROPMALFORMED", inferSchema='true', encoding="utf-8")
app_events = spark.read.csv(app_events_csv, header=True, mode="DROPMALFORMED", inferSchema='true', encoding="utf-8")
phone_brand_model = spark.read.csv("data/phone_brand_device_model.csv", header=True, mode="DROPMALFORMED", inferSchema='true')
phone_brand_model = phone_brand_model.dropDuplicates() # Drop the 523 rows that were duplicated in this df

events.show()   

+--------+--------------------+-------------------+---------+--------+
|event_id|           device_id|          timestamp|longitude|latitude|
+--------+--------------------+-------------------+---------+--------+
|       1|   29182687948017175|2016-05-01 00:55:25|   121.38|   31.24|
|       2|-6401643145415154744|2016-05-01 00:54:12|   103.65|   30.97|
|       3|-4833982096941402721|2016-05-01 00:08:05|    106.6|    29.7|
|       4|-6815121365017318426|2016-05-01 00:06:40|   104.27|   23.28|
|       5|-5373797595892518570|2016-05-01 00:07:18|   115.88|   28.66|
|       6| 1476664663289716375|2016-05-01 00:27:21|      0.0|     0.0|
|       7| 5990807147117726237|2016-05-01 00:15:13|   113.73|    23.0|
|       8| 1782450055857303792|2016-05-01 00:15:35|   113.94|    34.7|
|       9|-2073340001552902943|2016-05-01 00:15:33|      0.0|     0.0|
|      10|-8195816569128397698|2016-05-01 00:41:31|   119.34|   26.04|
|      11| 8663743929678393765|2016-05-01 00:44:13|   106.71|   39.51|
|     

In [94]:
events_tsf = events.select( \
                        col("event_id"),
                        col("device_id"),
                        hour(col("timestamp")).alias("hour"),
                        when(date_format("timestamp", 'E') == 'Mon', lit(1)).otherwise(lit(0)).alias("mon_count"),
                        when(date_format("timestamp", 'E') == 'Tue', 1).otherwise(0).alias("tue_count"),
                        when(date_format("timestamp", 'E') == 'Wed', 1).otherwise(0).alias("wed_count"),
                        when(date_format("timestamp", 'E') == 'Thu', 1).otherwise(0).alias("thu_count"),
                        when(date_format("timestamp", 'E') == 'Fri', 1).otherwise(0).alias("fri_count"),
                        when(date_format("timestamp", 'E') == 'Sat', 1).otherwise(0).alias("sat_count"),
                        when(date_format("timestamp", 'E') == 'Sun', 1).otherwise(0).alias("sun_count"),
                        when((date_format("timestamp", 'E') == 'Sat') | (date_format("timestamp", 'E') == 'Sun'), 1).otherwise(0).alias("weekend_count"),
                        when((date_format("timestamp", 'E') != 'Sat') & (date_format("timestamp", 'E') != 'Sun'), 1).otherwise(0).alias("weekday_count"),
                        when(date_format(col("timestamp"),"a") == "AM",1).otherwise(0).alias("am_count"),
                        when(date_format(col("timestamp"),"a") == "PM",1).otherwise(0).alias("pm_count"),
                        when(date_format(col("timestamp"),"HH") == 0,1).otherwise(0).alias("h0_count"),
                        when(date_format(col("timestamp"),"HH") == 1,1).otherwise(0).alias("h1_count"),
                        when(date_format(col("timestamp"),"HH") == 2,1).otherwise(0).alias("h2_count"),
                        when(date_format(col("timestamp"),"HH") == 3,1).otherwise(0).alias("h3_count"),
                        when(date_format(col("timestamp"),"HH") == 4,1).otherwise(0).alias("h4_count"),
                        when(date_format(col("timestamp"),"HH") == 5,1).otherwise(0).alias("h5_count"),
                        when(date_format(col("timestamp"),"HH") == 6,1).otherwise(0).alias("h6_count"),
                        when(date_format(col("timestamp"),"HH") == 7,1).otherwise(0).alias("h7_count"),
                        when(date_format(col("timestamp"),"HH") == 8,1).otherwise(0).alias("h8_count"),
                        when(date_format(col("timestamp"),"HH") == 9,1).otherwise(0).alias("h9_count"),
                        when(date_format(col("timestamp"),"HH") == 10,1).otherwise(0).alias("h10_count"),
                        when(date_format(col("timestamp"),"HH") == 11,1).otherwise(0).alias("h11_count"),
                        when(date_format(col("timestamp"),"HH") == 12,1).otherwise(0).alias("h12_count"),
                        when(date_format(col("timestamp"),"HH") == 13,1).otherwise(0).alias("h13_count"),
                        when(date_format(col("timestamp"),"HH") == 14,1).otherwise(0).alias("h14_count"),
                        when(date_format(col("timestamp"),"HH") == 15,1).otherwise(0).alias("h15_count"),
                        when(date_format(col("timestamp"),"HH") == 16,1).otherwise(0).alias("h16_count"),
                        when(date_format(col("timestamp"),"HH") == 17,1).otherwise(0).alias("h17_count"),
                        when(date_format(col("timestamp"),"HH") == 18,1).otherwise(0).alias("h18_count"),
                        when(date_format(col("timestamp"),"HH") == 19,1).otherwise(0).alias("h19_count"),
                        when(date_format(col("timestamp"),"HH") == 20,1).otherwise(0).alias("h20_count"),
                        when(date_format(col("timestamp"),"HH") == 21,1).otherwise(0).alias("h21_count"),
                        when(date_format(col("timestamp"),"HH") == 22,1).otherwise(0).alias("h22_count"),
                        when(date_format(col("timestamp"),"HH") == 23,1).otherwise(0).alias("h23_count")
                    ) \
                    .groupby("device_id") \
                    .agg( 
                        count("event_id").alias("events_per_device_count"),
                        min("hour").alias("min_hour"),
                        max("hour").alias("max_hour"),
                        func.sum("mon_count").alias("mon_count"),
                        func.sum("tue_count").alias("tue_count"),
                        func.sum("wed_count").alias("wed_count"),
                        func.sum("thu_count").alias("thu_count"),
                        func.sum("fri_count").alias("fri_count"),
                        func.sum("sat_count").alias("sat_count"),
                        func.sum("sun_count").alias("sun_count"),
                        func.sum("weekend_count").alias("weekend_count"),
                        func.sum("weekday_count").alias("weekday_count"),
                        func.sum("am_count").alias("am_count"),
                        func.sum("pm_count").alias("pm_count"),
                        func.sum("h0_count").alias("h0_count"),
                        func.sum("h1_count").alias("h1_count"),
                        func.sum("h2_count").alias("h2_count"),
                        func.sum("h3_count").alias("h3_count"),
                        func.sum("h4_count").alias("h4_count"),
                        func.sum("h5_count").alias("h5_count"),
                        func.sum("h6_count").alias("h6_count"),
                        func.sum("h7_count").alias("h7_count"),
                        func.sum("h8_count").alias("h8_count"),
                        func.sum("h9_count").alias("h9_count"),
                        func.sum("h10_count").alias("h10_count"),
                        func.sum("h11_count").alias("h11_count"),
                        func.sum("h12_count").alias("h12_count"),
                        func.sum("h13_count").alias("h13_count"),
                        func.sum("h14_count").alias("h14_count"),
                        func.sum("h15_count").alias("h15_count"),
                        func.sum("h16_count").alias("h16_count"),
                        func.sum("h17_count").alias("h17_count"),
                        func.sum("h18_count").alias("h18_count"),
                        func.sum("h19_count").alias("h19_count"),
                        func.sum("h20_count").alias("h20_count"),   
                        func.sum("h21_count").alias("h21_count"),
                        func.sum("h22_count").alias("h22_count"),
                        func.sum("h23_count").alias("h23_count")  
                     ) \
                     .persist()
events_tsf.show()

+--------------------+-----------------------+--------+--------+---------+---------+---------+---------+---------+---------+---------+-------------+-------------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+--------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+---------+
|           device_id|events_per_device_count|min_hour|max_hour|mon_count|tue_count|wed_count|thu_count|fri_count|sat_count|sun_count|weekend_count|weekday_count|am_count|pm_count|h0_count|h1_count|h2_count|h3_count|h4_count|h5_count|h6_count|h7_count|h8_count|h9_count|h10_count|h11_count|h12_count|h13_count|h14_count|h15_count|h16_count|h17_count|h18_count|h19_count|h20_count|h21_count|h22_count|h23_count|
+--------------------+-----------------------+--------+--------+---------+---------+---------+---------+---------+---------+---------+-------------+-------------+--------+-------

In [112]:
gender_age_events = gender_age_train.join(events, "device_id", "left_outer").persist()
gender_age_events_apps = gender_age_events.join(app_events, "event_id", "left_outer").persist()


device_apps_agg =  gender_age_events_apps.groupBy(gender_age_train["device_id"]) \
                               .agg(approx_count_distinct(app_events["app_id"]), approx_count_distinct(app_events["app_id"]) / approx_count_distinct(events["event_id"])) \
                               .toDF("device_id","apps_per_device_count","apps_per_event_avg")

device_active_apps_agg = gender_age_events_apps.filter("is_active=1").groupBy(gender_age_train["device_id"]) \
                               .agg(approx_count_distinct(app_events["app_id"]), approx_count_distinct(app_events["app_id"]) / approx_count_distinct(events["event_id"])) \
                               .toDF("device_id","apps_active_per_device_count","apps_active_per_event_avg")   

print device_active_apps_agg.show()
train = genderAgeEvents.join(label_lookup, "group","left_outer")
                     .join(events_tsf, "device_id", "left_outer")
                     .join(device_apps_agg, "device_id", "left_outer")
                     .join(device_active_apps_agg, "device_id", "left_outer")
                     .join(phone_brand_model, "device_id", "left_outer")
                     .select("device_id", "label"
                             ,"events_per_device_count","min_hour","max_hour","mon_count","tue_count","wed_count","thu_count","fri_count","sat_count","sun_count","weekend_count","weekday_count"
                             ,"am_count","pm_count","h0_count","h1_count","h2_count","h3_count","h4_count","h5_count","h6_count","h7_count","h8_count","h9_count","h10_count"
                             ,"h11_count","h12_count","h13_count","h14_count","h15_count","h16_count","h17_count","h18_count","h19_count","h20_count","h21_count","h22_count","h23_count"
                             ,"apps_per_device_count","apps_per_event_avg"
                             ,"appsActive_per_device_count","apps_active_per_event_avg"
                             ,"brand_model_key")
                     .na.fill(0)


+--------------------+----------------------------+-------------------------+
|           device_id|apps_active_per_device_count|apps_active_per_event_avg|
+--------------------+----------------------------+-------------------------+
| 7076350749443045918|                           5|      0.08620689655172414|
|-4845339526909437808|                          74|       0.5362318840579711|
|-1463646610464196027|                          16|       0.9411764705882353|
| 4892402731847362338|                          10|      0.06711409395973154|
| 6673421738489059698|                          26|        1.368421052631579|
|  447514480517228982|                          18|                      0.6|
| 3521390525163236174|                           2|      0.08333333333333333|
|-2058385977734641672|                          31|                      0.5|
|-2157032628966288294|                          12|                      2.4|
|-4296584415415502143|                           2|      0.05263

# Prepare Test Data